In [ ]:
import itertools as it
import operator
import re
import subprocess
import tempfile
import time
from collections import Counter
from pathlib import Path

import gfapy
import holoviews as hv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.compute as pc
import pyfastx
from pyarrow import csv
from tqdm.auto import tqdm, trange

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.cloning.design as design
import paulssonlab.cloning.sequence as sequence
import paulssonlab.sequencing.io as sio

In [ ]:
hv.extension("bokeh")

# Functions

# Config

In [ ]:
# data_dir = Path("/home/jqs1/scratch/sequencing/230707_repressilators/20230707_2040_MN35044_FAS94231_25542e0d/_temp/")
# carlos_variants = pyfastx.Fasta(data_dir / "230726_carlos/Savinov_Fragment_key.fasta")

# Test

In [ ]:
# SEE: https://github.com/maickrau/GraphAligner/issues/18#issuecomment-635793833

In [ ]:
# TODO: REPLACE RANDOM_BASES, use rng?

In [ ]:
# segments = [["aa", "cc"], ["tttat", "cccgc"], ["tccccccc"]]
# segments = [["aa"*20, "cc"*20], ["tttat"*5, "cccgc"*5], ["tccccccc"*5]]
# segments = [
#     [design.random_bases(40) for i in range(2)],
#     [design.random_bases(40) for i in range(2)],
#     [design.random_bases(40) for i in range(2)],
# ]
segments = [
    [design.random_bases(40, "atcg") for i in range(2)],
    [design.random_bases(40, "atcg") for i in range(2)],
    [design.random_bases(10, "atcg") for i in range(10)],
    [design.random_bases(40, "atcg") for i in range(2)],
    [design.random_bases(40, "atcg") for i in range(2)],
]

In [ ]:
%%time
# base_args = ["-b", "100000", "-C", "500000", "--verbose"]
# base_args = ["--verbose", "--seeds-clustersize", "0"]
base_args = ["--verbose"]
graphaligner_args = [
    # [*base_args, "-x", "vg"],
    [
        *base_args,
        "-b",
        "100000",
        "-C",
        "500000",
        "--seedless-DP",
        "--DP-restart-stride",
        "1000",
    ],
    # [*base_args, "-x", "dbg"],
    # [*base_args, "-b", "15", "-C", "-1", "--seeds-minimizer-windowsize", "20", "--seeds-minimizer-density", "0", "--seeds-minimizer-length", "10"],
    # [
    #     *base_args,
    #     "--seeds-mxm-windowsize",
    #     "500",
    #     "--seeds-mxm-length",
    #     "30",
    #     "--seeds-mem-count",
    #     "10000",
    #     "--bandwidth",
    #     "15",
    #     # "--multimap-score-fraction",
    #     # "0.99",
    #     # "--precise-clipping",
    #     # "0.85",
    #     "--min-alignment-score",
    #     "5000",
    #     # "--clip-ambiguous-ends",
    #     # "100",
    #     # "--overlap-incompatible-cutoff",
    #     # "0.15",
    #     "--max-trace-count",
    #     "5",
    # ],
    # [*base_args, "-b", "15", "-C", "-1", "--seeds-mum-count", "-1"],
    # [*base_args, "-b", "15", "-C", "-1", "--seeds-mem-count", "-1"],
    # [
    #     *base_args,
    #     "-b",
    #     "15",
    #     "-C",
    #     "-1",
    #     "--seeds-mum-count",
    #     "-1",
    #     "--max-trace-count",
    #     "-1",
    #     "--max-cluster-extend",
    #     "-1",
    # ],
    # [
    #     *base_args,
    #     "-b",
    #     "15",
    #     "-C",
    #     "-1",
    #     "--seeds-mem-count",
    #     "-1",
    #     "--max-trace-count",
    #     "-1",
    #     "--max-cluster-extend",
    #     "-1",
    # ],
    # [
    #     *base_args,
    #     # "--seedless-DP",
    #     # "--DP-restart-stride",
    #     # "1000",
    #     # "--precise-clipping",
    #     # "0.502",
    #     # "--X-drop",
    #     # "1000000",
    # ],
]
res, ground_truth = run_aligner_synthetic(
    segments,
    num_reads=100,
    args=graphaligner_args,
    q=10,
    rng=np.random.default_rng(714),
)
errors = [check_alignment(run[0], ground_truth) for run in res]
times = [run[1] for run in res]

In [ ]:
res[0][0]["path"]

In [ ]:
for t, run_errors, run_args in zip(times, errors, graphaligner_args):
    print(f"{' '.join(run_args).rjust(80)}: {len(run_errors)} ({t:.2f}s)")

In [ ]:
res

In [ ]:
ground_truth

In [ ]:
res[0][0].column("path")